In [1]:
import numpy as np

# Parâmetros do problema
n_baias = 10
N = 100
alfa = 0.5
beta = 0.5

def inicializar(B, T):
    for i in range(n_baias):
        for j in range(n_baias):
            if i != j:
                B[i][j] = np.random.randint(0, 2)
    for i in range(n_baias):
        T[i] = np.random.randint(0, 100)

def avaliar_solucao(s, B, T, N):
    p = 1
    nmov = 0
    instab = 0.0
    while p < N:
        rc, rd = extrair_regras(s, p)
        if p > 1:
            instab += calc_DX_DZ(B)
            aux, B, T = descarregar(rd, B, p, T)
            nmov += aux
        if p < N - 1:
            aux, B, T = carregar(rc, B, T, p)
            instab += calc_DX_DZ(B)
            nmov += aux
        p += 1
    return alfa * nmov + beta * instab

def extrair_regras(s_i, p):
    rc = s_i[p][:n_baias]
    rd = s_i[p][n_baias:]
    return rc, rd

def calc_DX_DZ(B):
    return np.sum(B)

def carregar(rc, B, T, p):
    mov = 0
    for i in range(n_baias):
        if rc[i] == 1:
            menor = np.argmin(T)
            B[menor][i] = 1
            T[menor] += p
            mov += 1
    return mov, B, T

def descarregar(rd, B, p, T):
    mov = 0
    for i in range(n_baias):
        if rd[i] == 1:
            maior = np.argmax(T)
            B[i][maior] = 0
            T[maior] -= p
            mov += 1
    return mov, B, T

def selecao_roleta(populacao, fitness):
    selecionados = []
    for _ in range(len(populacao)):
        pai1_idx = roleta(fitness)
        pai2_idx = roleta(fitness)
        selecionados.append((populacao[pai1_idx], populacao[pai2_idx]))
    return selecionados

def roleta(fitness):
    total_fitness = np.sum(fitness)
    escolhido = np.random.uniform(0, total_fitness)
    soma = 0
    for i, f in enumerate(fitness):
        soma += f
        if soma >= escolhido:
            return i

def crossover(pai1, pai2):
    ponto_corte = np.random.randint(0, len(pai1))
    filho1 = np.concatenate((pai1[:ponto_corte], pai2[ponto_corte:]))
    filho2 = np.concatenate((pai2[:ponto_corte], pai1[ponto_corte:]))
    return filho1, filho2

def mutacao(individuo, taxa_mutacao):
    for i in range(len(individuo)):
        if np.random.uniform(0, 1) < taxa_mutacao:
            individuo[i] = 1 - individuo[i]
    return individuo

# Parâmetros do algoritmo genético
tamanho_populacao = 20
taxa_mutacao = 0.1
num_geracoes = 10

# Inicialização da população
populacao = np.random.randint(2, size=(tamanho_populacao, N, 2 * n_baias))

# Inicialização da matriz de conexões e vetor de tempos
B = np.zeros((n_baias, n_baias))
T = np.zeros(n_baias)
inicializar(B, T)

# Avaliação inicial da população
fitness = np.array([avaliar_solucao(s, B, T, N) for s in populacao])

# Execução das gerações
for geracao in range(num_geracoes):
    # Realizar seleção dos indivíduos para a próxima geração
    selecionados = selecao_roleta(populacao, fitness)
    
    # Realizar crossover
    nova_populacao = []
    for pais in selecionados:
        filho1, filho2 = crossover(pais[0], pais[1])
        nova_populacao.append(filho1)
        nova_populacao.append(filho2)
    
    # Realizar mutação
    nova_populacao = [mutacao(individuo, taxa_mutacao) for individuo in nova_populacao]
    
    # Atualizar a população
    populacao = np.array(nova_populacao)
    
    # Avaliar a nova população
    fitness = np.array([avaliar_solucao(s, B, T, N) for s in populacao])

# Encontrar a melhor solução
melhor_idx = np.argmin(fitness)
melhor_solucao = populacao[melhor_idx]

print("Melhor solução encontrada:")
print(melhor_solucao)
print("Fitness da melhor solução:", fitness[melhor_idx])


KeyboardInterrupt: 